In [5]:
import pandas as pd
import numpy as np
from scipy.stats import iqr

from ppc_db import sql_extractdata

db_args = {
    'user':'root',
    'password':'password',
    'host':'localhost',
    'port':3306,
    'database':'CSV_DB 7'
 }

In [19]:
import datetime

cols = ['MONTH(date)', '7_county', 'COUNT(`tx_id`)']

query = f"SELECT {", ".join(str(x) for x in cols)} FROM Sep24pricepaidcomplete WHERE MONTH(date) < 8 AND YEAR(date) = 2024 GROUP BY MONTH(date), 7_county"
result = pd.DataFrame(sql_extractdata(query, **db_args),columns=cols)

df_Sep = (result.join
      (pd.read_csv('county_to_region.csv',
                   index_col='7_county',
                   dtype=str),
                   on='7_county')
     )

query = f"SELECT {", ".join(str(x) for x in cols)} FROM Oct24pricepaidcomplete WHERE MONTH(date) < 8 AND YEAR(date) = 2024 GROUP BY MONTH(date), 7_county"
result = pd.DataFrame(sql_extractdata(query, **db_args),columns=cols)

df_Oct = (result.join
      (pd.read_csv('county_to_region.csv',
                   index_col='7_county',
                   dtype=str),
                   on='7_county')
     )

In [35]:
df_Oct[df_Oct['region']=='London']#.join(df_Sep[df_Sep['region']=='London'],on='MONTH(date)')

,MONTH(date),7_county,COUNT(`tx_id`),region
11,7,GREATER LONDON,3724,London
12,4,GREATER LONDON,5627,London
13,3,GREATER LONDON,6034,London
14,6,GREATER LONDON,3766,London
15,5,GREATER LONDON,5408,London
16,1,GREATER LONDON,5350,London
17,2,GREATER LONDON,5587,London


In [36]:
df_Sep= df_Sep.drop('7_county',axis=1).groupby(['MONTH(date)','region']).sum().unstack()['COUNT(`tx_id`)']
df_Oct = df_Oct.drop('7_county',axis=1).groupby(['MONTH(date)','region']).sum().unstack()['COUNT(`tx_id`)']

In [39]:
# First Quarter Reliable

(df_Oct-df_Sep)/df_Sep

region,East Midlands,East of England,London,North East,North West,South East,South West,Wales,West Midlands,Yorkshire and the Humber
MONTH(date),,,,,,,,,,
1,0.034181,0.036488,0.026674,0.030603,0.030226,0.032530,0.025931,0.022069,0.026644,0.035698
2,0.013129,0.015966,0.017113,0.011540,0.013721,0.015058,0.011909,0.012997,0.012886,0.010618
3,0.030154,0.026674,0.034814,0.021394,0.035160,0.028601,0.028443,0.032793,0.034855,0.030077
4,0.237232,0.215715,0.242438,0.179178,0.220238,0.209916,0.173021,0.190431,0.214149,0.197231
5,0.236539,0.275551,0.340605,0.250841,0.241386,0.278041,0.259811,0.251369,0.274949,0.229418
6,0.040450,0.040295,0.054016,0.045003,0.049377,0.042481,0.031907,0.056561,0.046959,0.048282
7,0.140364,0.119595,0.146199,0.120104,0.142192,0.120676,0.108842,0.193844,0.145462,0.129990
